## Optimizer Code

### Useful libraries
[CVXPY Convex Optimizer](https://www.cvxpy.org/)

[NumPy Scientific Package](https://numpy.org/)

[Matplotlib Pyplot Python Module](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html?highlight=pyplot#module-matplotlib.pyplot)

[Scipy Linear Algebra Functions](https://docs.scipy.org/doc/scipy/reference/linalg.html)

In [1]:
# Import packages.
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg

import import_ipynb
import utils

### Optimizer Function

`def optimizer(p_i, p_f, robots, radius)`

Receding Horizon Path Planning Solver

#### Parameters:
* **p_i&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Start position
* **p_f&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Goal position
* **robots&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of lists of floats**
    * The position of robots in front
* **radius&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;float**
    * Safety radius
* **plot_flag&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * True to plot the construction of the optimal value
* **verbose&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;bool**
    * True to check the code

#### Return:
* **optimal_value&nbsp;&nbsp;&nbsp;:&nbsp;&nbsp;&nbsp;ndarray of floats**
    * Optimal solution of QP Problem

In [20]:
def optimizer(p_i, p_f, robots, radius, plot_flag, verbose):
    np.random.seed(1)
    # Dimensions
    T = 20 # Steps
    m = 2 # u dimension
    n = 2 # x dimension
    # Mechanics
    alpha = 0.2
    A = np.eye(n) + alpha*np.random.randn(n,n)
    B = np.random.randn(n,m)
    # Parameters
    R_sqrt = cp.Parameter((m,m), PSD = True)
    Q_sqrt = cp.Parameter((n,n), PSD = True)
    Qf_sqrt = cp.Parameter((n,n), PSD = True)
    R_sqrt.value = np.identity(m)
    Q_sqrt.value = np.identity(n)
    Qf_sqrt.value = np.identity(n)
    # Variables
    p = cp.Variable((n,T+1))
    u = cp.Variable((m,T))
    u_max_x = 1
    u_max_y = 1
    # Artificial operations for DPP
    R_sim = np.random.randn(m,m)
    Q_sim = np.random.randn(n,n)
    Qf_sim = np.random.randn(n,n)
    R_inst = R_sim.T @ R_sim
    Q_inst = Q_sim.T @ Q_sim
    Qf_inst = Qf_sim.T @ Qf_sim
    # Cost and Constraints
    cost = 0
    constr = []
    constr += [p[:,0] == p_i]
    for t in range(T):
        cost += cp.sum_squares(Q_sqrt @ p[:,t]-p_f) + cp.sum_squares(R_sqrt @
                                                                     u[:,t])
        constr += [p[:,t+1] == A@p[:,t] + B@u[:,t],
                   cp.abs(u[0,t]) <= u_max_x,
                   cp.abs(u[1,t]) <= u_max_y]                                                        
        for robot in robots:
            p_ij = (robot - p_i)
            temp = (cp.transpose(p[:,t] - (p_i + robot)/2) @ p_ij
                    + (radius * np.linalg.norm(p_ij)) <= 0)
            #print(temp)
            constr += [temp]
    # Last case
    cost += cp.sum_squares(Qf_sqrt @ p[:,T]-p_f)
    for robot in robots:
        p_ij = (robot - p_i)
        constr += [(cp.transpose(p[:,T] - (p_i + robot)/2) @ p_ij
                    + (radius * np.linalg.norm(p_ij)) <= 0)]
    #Assing values
    R_sqrt.value = scipy.linalg.sqrtm(R_inst)
    Q_sqrt.value = scipy.linalg.sqrtm(Q_inst)
    Qf_sqrt.value = scipy.linalg.sqrtm(Q_inst) #For accuracy
    # Objective
    prob = cp.Problem(cp.Minimize(cost), constr)
    # Solve with no Disciplined Geometric Programming
    #prob.solve(solver=cp.ECOS, gp=False, verbose=True)
    prob.solve(solver=cp.ECOS, gp=False)
    # Optimal solution
    optimal_value = prob.value
    #Graphics
    if plot_flag:
       # Plot results.
        %matplotlib inline
        %config InlineBackend.figure_format = 'svg'
        f = plt.figure()
        # Plot (u_t)_1.
        ax = f.add_subplot(411)
        plt.plot(u[0,:].value)
        plt.ylabel(r"$(u_t)_1$", fontsize=16, color='black')
        plt.yticks(np.linspace(-1.0, 1.0, 3), color='black')
        plt.xticks([])
        # Plot (u_t)_2.
        plt.subplot(4,1,2)
        plt.plot(u[1,:].value)
        plt.ylabel(r"$(u_t)_2$", fontsize=16, color='black')
        plt.yticks(np.linspace(-1, 1, 3), color='black')
        plt.xticks([])
        # Plot (x_t)_1.
        plt.subplot(4,1,3)
        p1 = p[0,:].value
        plt.plot(p1)
        plt.ylabel(r"$(p_t)_1$", fontsize=16, color='black')
        plt.yticks([-6, 0, 6], color='black')
        plt.ylim([-6, 6])
        plt.xticks([])
        # Plot (x_t)_2.
        plt.subplot(4,1,4)
        p2 = p[1,:].value
        plt.plot(range(21), p2)
        plt.yticks([-6, 0, 6], color='black')
        plt.ylim([-6, 6])
        plt.ylabel(r"$(p_t)_2$", fontsize=16, color='black')
        plt.xlabel(r"$t$", fontsize=16, color='black')
        plt.xticks(color='black')
        plt.tight_layout()
        plt.savefig('/home/leduin/Escritorio/Tesis/Resultados/Imagenes/solver_example.svg')
        plt.close()
    # Check status
    #print("We get the ", prob.status, " value")
    # Return
    if verbose:
        print(prob.status)
    return p.value[:,T]

In [ ]:
def aga(pos_i, pos_f, bvc_vrtx, closer_edges):
    edge_points = utils.perpendicular_dist(pos_f, bvc_vrtx)
    target_point = np.array(pos_f)
    vertices = np.array(bvc_vrtx + edge_points)
    # Initialize
    closest_point = np.linalg.norm(vertices[0] - target_point)
    pointer = 0
    # Review the rest of vertices
    for j in range(1, len(vertices)):
        # Measure the distance of the candidate vertex to the target
        candidate = np.linalg.norm(vertices[j] - target_point)
        # Take the closest point
        if candidate < closest_point:
            closest_point = candidate
            pointer = j
    # Update the closest point 
    closest_point = vertices[pointer]
    get_intersection = False
    k = 0
    while not get_intersection and k < len(bvc_vrtx)-1:
        closest_point = utils.sharpen(pos_f, pos_i, bvc_vrtx[k+1], bvc_vrtx[k])
        is_in_edge = utils.between(bvc_vrtx[k+1], bvc_vrtx[k], closest_point)
        is_in_path = utils.between(pos_f, pos_i, closest_point)
        if is_in_edge and is_in_path:
            get_intersection = True
            closer_edges.extend([bvc_vrtx[k+1], bvc_vrtx[k]])
        else:
            k += 1
    if not get_intersection:
        closer_edges.extend([[0,0],[0,0]])
    return closer_edges, closest_point

In [ ]:
def rhc(neighbors, index, pos, opt, radius):
    front_robots = []
    for in_front in neighbors:
        if index == in_front[0]:
            front_robots.append(pos["i"][in_front[1]])
        elif index == in_front[1]:
            front_robots.append(pos["i"][in_front[0]])
    if opt["verbose"]:
        print("Robots in front:")
        for k in front_robots:
            print(k)
    # Calculate the closest point using the CVXPY Solver
    closest_point = optimizer(pos["i"][count], pos["f"][count],
                              np.array(front_robots), radius, False,
                              opt["verbose"])
    get_intersection = False
    i = 0
    while not get_intersection and i < len(bvc_vrtx)-1:
        if utils.between(bvc_vrtx[i+1], bvc_vrtx[i], closest_point):
            get_intersection = True
            closer_edges.extend([bvc_vrtx[i+1], bvc_vrtx[i]])
        else:
            i += 1
    if not get_intersection:
        closer_edges.extend([[0,0],[0,0]])
    return closer_edges, closest_point